In [1]:
import cv2
import pyautogui
import time
from math import hypot
import mediapipe as mp
import matplotlib.pyplot as plt
from directkeys import PressKey, ReleaseKey,W,A,S,D
import pydirectinput

In [2]:
mp_pose = mp.solutions.pose

pose_image = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5,model_complexity=1)

pose_video = mp_pose.Pose(static_image_mode=False, model_complexity=1 ,min_tracking_confidence=0.7)

mp_drawing = mp.solutions.drawing_utils

In [3]:
def detectPose(image, pose, draw = False, display =False):

    output_image = image.copy()
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(imageRGB)

    if results.pose_landmarks and draw:
        mp_drawing.draw_landmarks(image = output_image, landmark_list = results.pose_landmarks, connections = mp_pose.POSE_CONNECTIONS,
                                 landmark_drawing_spec=mp_drawing.DrawingSpec(color = (255,255,255),thickness =3, circle_radius =3),
                                 connection_drawing_spec = mp_drawing.DrawingSpec(color = (49,125,237),thickness =2, circle_radius = 2))

    if display:
        plt.figure(figsize = [22,22])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
    else:
        return output_image, results

In [4]:
def checkAccDec(image , results , draw = False , display = False):
    
    
    height, width, _ = image.shape
    
    # Create a copy of the input image to write the hands status label on.
    output_image = image.copy()
    
    left_line = (width//2) -190
    right_line = (width//2)  + 190
    
    
    height_right_wrist = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y * height)
    
    if(height_right_wrist<450):
        posture="ACCELERATE"
    elif(height_right_wrist>450 and height_right_wrist<720):
        posture="DECELERATE"
    else:
        posture="NIL"
    
    if draw:
        cv2.putText(output_image, posture , (5, height - 50), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
        cv2.line(output_image, (left_line,0),(left_line,height),(255, 255, 255), 2)
        cv2.line(output_image, (right_line,0),(right_line,height),(255, 255, 255), 2)
        cv2.line(output_image, (0, 450),(width, 450),(255, 255, 255), 2)
    
    if display:

        # Display the output image.
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
    
    # Otherwise
    else:
    
        # Return the output image and posture indicating whether the person is standing straight or has jumped, or crouched.
        return output_image, posture
    
    
    


In [5]:
# camera_video = cv2.VideoCapture(0)
# camera_video.set(3,1280)
# camera_video.set(4,960)
 
# # Create named window for resizing purposes.
# cv2.namedWindow('Verticial Movements', cv2.WINDOW_NORMAL)
 
# # Iterate until the webcam is accessed successfully.
# while camera_video.isOpened():
    
#     # Read a frame.
#     ok, frame = camera_video.read()
    
#     # Check if frame is not read properly then continue to the next iteration to read the next frame.
#     if not ok:
#         continue
    
#     # Flip the frame horizontally for natural (selfie-view) visualization.
#     frame = cv2.flip(frame, 1)
    
#     # Get the height and width of the frame of the webcam video.
#     frame_height, frame_width, _ = frame.shape
    
#     # Perform the pose detection on the frame.
#     frame, results = detectPose(frame, pose_video, draw=True)
    
#     # Check if the pose landmarks in the frame are detected.
#     if results.pose_landmarks:
            
#         # Check the posture (jumping, crouching or standing) of the person in the frame. 
#         frame, _ = checkAccDec(frame, results, draw=True)
                
#     # Display the frame.
#     cv2.imshow('ACC OR DEC', frame)
    
#     # Wait for 1ms. If a a key is pressed, retreive the ASCII code of the key.
#     k = cv2.waitKey(1) & 0xFF
    
#     # Check if 'ESC' is pressed and break the loop.
#     if(k == 27):
#         break
 
# # Release the VideoCapture Object and close the windows.
# camera_video.release()
# cv2.destroyAllWindows()

In [5]:
def checkLeftRight(image, results, draw = False, display = False):
    
    # Get the height and width of the image.
    height, width, _ = image.shape
    
    # Create a copy of the input image to write the horizontal position on.
    output_image = image.copy()
    
    posture = "posture"
    
    left_x = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * width)
 
    right_x = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * width)
    
    
    
    center = (left_x+right_x)//2
    
    left_line = (width//2) -190
    right_line = (width//2)  + 190
    #left_line_extreme = (width//2) -230
    #right_line_extreme = (width//2)  + 230
    
    if left_x <= left_line:
        posture = "Left"
    elif right_x >= right_line:
        posture = "Right"
    else:
        posture = "Normal Front"
            
    if draw:
        cv2.putText(output_image, posture, (5, height - 50), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
        cv2.line(output_image, (center,0),(center,height),(255, 255, 255), 2)
        cv2.line(output_image, (left_line,0),(left_line,height),(255, 255, 255), 2)
        cv2.line(output_image, (right_line,0),(right_line,height),(255, 255, 255), 2)
        #cv2.line(output_image, (left_line_extreme,0),(left_line_extreme,height),(255, 255, 255), 2)
        #cv2.line(output_image, (right_line_extreme,0),(right_line_extreme,height),(255, 255, 255), 2)
          
        
    if display:
        plt.figure(figsize= [10,10])
        
        plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        plt.show() # added this line to plot
    
    else:
        
        return output_image, posture

In [7]:
# camera_video = cv2.VideoCapture(0)
# camera_video.set(3,1280)
# camera_video.set(4,960)
 
# # Create named window for resizing purposes.
# cv2.namedWindow('lr Movements', cv2.WINDOW_NORMAL)
 
# # Iterate until the webcam is accessed successfully.
# while camera_video.isOpened():
    
#     # Read a frame.
#     ok, frame = camera_video.read()
    
#     # Check if frame is not read properly then continue to the >next iteration to read the next frame.
#     if not ok:
#         continue
    
#     # Flip the frame horizontally for natural (selfie-view) visualization.
#     frame = cv2.flip(frame, 1)
    
#     # Get the height and width of the frame of the webcam video.
#     frame_height, frame_width, _ = frame.shape
    
#     # Perform the pose detection on the frame.
#     frame, results = detectPose(frame, pose_video, draw=True)
    
#     # Check if the pose landmarks in the frame are detected.
#     if results.pose_landmarks:
            
#         # Check the posture (jumping, crouching or standing) of the person in the frame. 
#         frame, _ = checkLeftRight(frame, results, draw=True)
                
#     # Display the frame.
#     cv2.imshow('LEFT OR RIGHT', frame)
    
#     # Wait for 1ms. If a a key is pressed, retreive the ASCII code of the key.
#     k = cv2.waitKey(1) & 0xFF
    
#     # Check if 'ESC' is pressed and break the loop.
#     if(k == 27):
#         break
 
# # Release the VideoCapture Object and close the windows.
# camera_video.release()
# cv2.destroyAllWindows()

In [6]:
def checkHandsJoined(image, results, draw=False, display=False):
    '''
    This function checks whether the hands of the person are joined or not in an image.
    Args:
        image:   The input image with a prominent person whose hands status (joined or not) needs to be classified.
        results: The output of the pose landmarks detection on the input image.
        draw:    A boolean value that is if set to true the function writes the hands status &amp; distance on the output image. 
        display: A boolean value that is if set to true the function displays the resultant image and returns nothing.
    Returns:
        output_image: The same input image but with the classified hands status written, if it was specified.
        hand_status:  The classified status of the hands whether they are joined or not.
    '''
    
    # Get the height and width of the input image.
    height, width, _ = image.shape
    
    # Create a copy of the input image to write the hands status label on.
    output_image = image.copy()
    
    # Get the left wrist landmark x and y coordinates.
    left_wrist_landmark = (results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].x * width,
                          results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y * height)
 
    # Get the right wrist landmark x and y coordinates.
    right_wrist_landmark = (results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].x * width,
                           results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y * height)
    
    # Calculate the euclidean distance between the left and right wrist.
    euclidean_distance = int(hypot(left_wrist_landmark[0] - right_wrist_landmark[0],
                                   left_wrist_landmark[1] - right_wrist_landmark[1]))
    
    # Compare the distance between the wrists with a appropriate threshold to check if both hands are joined.
    if euclidean_distance<130:
        
        # Set the hands status to joined.
        hand_status = 'Hands Joined'
        
        # Set the color value to green.
        color = (0, 255, 0)
        
    # Otherwise.    
    else:
        
        # Set the hands status to not joined.
        hand_status = 'Hands Not Joined'
        
        # Set the color value to red.
        color = (0, 0, 255)
        
    # Check if the Hands Joined status and hands distance are specified to be written on the output image.
    if draw:
 
        # Write the classified hands status on the image. 
        cv2.putText(output_image, hand_status, (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, color, 3)
        
        # Write the the distance between the wrists on the image. 
        cv2.putText(output_image, f'Distance: {euclidean_distance}', (10, 70),
                    cv2.FONT_HERSHEY_PLAIN, 2, color, 3)
        
    # Check if the output image is specified to be displayed.
    if display:
 
        # Display the output image.
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
    
    # Otherwise
    else:
    
        # Return the output image and the classified hands status indicating whether the hands are joined or not.
        return output_image, hand_status 

In [9]:
# camera_video = cv2.VideoCapture(0)
# camera_video.set(3,1280)
# camera_video.set(4,960)
 
# # Create named window for resizing purposes.
# cv2.namedWindow('Asphalt 9 Pose Detection', cv2.WINDOW_NORMAL)

# # Initialize a variable to store the time of the previous frame.
# time1 = 0

# # Initialize a variable to store the state of the game (started or not).
# game_started = False  


# num_of_frames = 10

# #currentKey = list()

# last_left_press_time = time.time()
# last_right_press_time = time.time()

# while camera_video.isOpened():
    
#     # Read a frame.
#     ok, frame = camera_video.read()
#     ok, frame1 = camera_video.read()
    
#     # Check if frame is not read properly then continue to the next iteration to read the next frame.
#     if not ok:
#         continue
        
#     frame = cv2.flip(frame, 1)
#     frame1 = cv2.flip(frame1,1)
    
#     # Get the height and width of the frame of the webcam video.
#     frame_height, frame_width, _ = frame.shape
    
#     # Perform the pose detection on the frame.
#     frame, results = detectPose(frame, pose_video, draw=game_started)
#     frame1, results = detectPose(frame1, pose_video,draw=False)
    
#     key=False
        
#     if results.pose_landmarks:
        
#         # Check if the game has started
#         if game_started:
            
            
#             frame, accordec = checkAccDec(frame, results, draw=True)
#             frame1 ,lor = checkLeftRight(frame1, results, draw = False)
#             #key = False

        
#             if accordec == "ACCELERATE":
#                 PressKey(W)
#             else:
#                 ReleaseKey(W)

#             if accordec == "DECELERATE":
#                 PressKey(S)
#             else:
#                 ReleaseKey(S)

        
#             if lor == "Left" and (time.time() - last_left_press_time) >= 0.5:
#                 PressKey(A)
#                 last_left_press_time = time.time()
           

#             if lor == "Right" and (time.time() - last_right_press_time) >= 0.5:
#                 PressKey(D)
#                 last_right_press_time = time.time()
             
                 
#         else:
            
#             # Write the text representing the way to start the game on the frame. 
#             cv2.putText(frame, 'JOIN BOTH HANDS TO START THE GAME.', (5, frame_height - 10), cv2.FONT_HERSHEY_PLAIN,
#                         2, (0, 255, 0), 3)
        
      
#         if checkHandsJoined(frame, results)[1] == 'Hands Joined':
#             counter += 1
#              #counter is used to give the base time after which the detection should be started 
#             # Check if the counter is equal to the required number of consecutive frames.  
    
#             if counter == num_of_frames:
                
#                 # Command to Start the game first time.
#                 #----------------------------------------------------------------------------------------------------------
#                 # Check if the game has not started yet.
#                 if not(game_started):
#                     counter=0
#                     # Update the value of the variable that stores the game state.
#                     game_started = True
#                     pyautogui.press(keys='enter', presses=2)
#                 else:
#                     counter=0
#                     #pyautogui.press()
#                     game_started = False
#         else:
#             counter=0
            
#         time2 = time.time()
    
#         # Check if the difference between the previous and this frame time &gt; 0 to avoid division by zero.
#         if (time2 - time1) > 0:
    
#         # Calculate the number of frames per second.
#             frames_per_second = 1.0 / (time2 - time1)
        
#         # Write the calculated number of frames per second on the frame.
#             cv2.putText(frame, 'FPS: {}'.format(int(frames_per_second)), (10, 30),cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)
    
#     # Update the previous frame time to this frame time.
#     # As this frame will become previous frame in next iteration.
#         time1 = time2
    
#     #----------------------------------------------------------------------------------------------------------------------
    
#     # Display the frame.            
#         cv2.imshow('Asphalt 9 Pose Detection', frame)
        
        
#         #if not key and len(currentKey) != 0:
#           #  for current in currentKey:
#           #      ReleaseKey(current)
#          #   currentKey = list()

    
#     # Wait for 1ms. If a a key is pressed, retreive the ASCII code of the key.
#         k = cv2.waitKey(1) & 0xFF    
    
#     # Check if 'ESC' is pressed and break the loop.
#         if(k == 27):
#             break
 
# # Release the VideoCapture Object and close the windows.                  
# camera_video.release()
# cv2.destroyAllWindows()

In [8]:
camera_video = cv2.VideoCapture(0)
camera_video.set(3, 1280)
camera_video.set(4, 960)

cv2.namedWindow('Asphalt 9 Pose Detection', cv2.WINDOW_NORMAL)

time1 = 0
game_started = False

num_of_frames = 10
counter = 0

last_left_press_time = time.time()
last_right_press_time = time.time()

while camera_video.isOpened():
    ok, frame = camera_video.read()
    ok, frame1 = camera_video.read()

    if not ok:
        continue

    frame = cv2.flip(frame, 1)
    frame1 = cv2.flip(frame1, 1)

    frame_height, frame_width, _ = frame.shape
    frame, results = detectPose(frame, pose_video, draw=game_started)
    frame1, results = detectPose(frame1, pose_video, draw=False)

    key = False

    if results.pose_landmarks:

        if game_started:

            frame, accordec = checkAccDec(frame, results, draw=True)
            frame1, lor = checkLeftRight(frame1, results, draw=False)

            if accordec == "ACCELERATE":
                PressKey(W)
            else:
                ReleaseKey(W)

            if accordec == "DECELERATE":
                PressKey(S)
            else:
                ReleaseKey(S)

            if lor == "Left" and (time.time() - last_left_press_time) >= 0.5:
                PressKey(A)
                last_left_press_time = time.time()

            if lor == "Right" and (time.time() - last_right_press_time) >= 0.5:
                PressKey(D)
                last_right_press_time = time.time()

        else:

            cv2.putText(frame, 'JOIN BOTH HANDS TO START THE GAME.', (5, frame_height - 10),
                        cv2.FONT_HERSHEY_PLAIN,
                        2, (0, 255, 0), 3)

        if checkHandsJoined(frame, results)[1] == 'Hands Joined':
            counter += 1

            if counter == num_of_frames:
                if not (game_started):
                    counter = 0
                    game_started = True
                    pyautogui.press(keys='enter', presses=2)
                else:
                    counter = 0
                    game_started = False
        else:
            counter = 0

        time2 = time.time()

        if (time2 - time1) > 0:
            frames_per_second = 1.0 / (time2 - time1)
            cv2.putText(frame, 'FPS: {}'.format(int(frames_per_second)), (10, 30),
                        cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)

    time1 = time2

    cv2.imshow('Asphalt 9 Pose Detection', frame)

    k = cv2.waitKey(1) & 0xFF

    if (k == 27):
        break

camera_video.release()
cv2.destroyAllWindows()